In [1]:
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import matplotlib.pyplot as plt
from siphon.catalog import TDSCatalog
import xarray as xr
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import ipywidgets as widgets

In [2]:
new_gfs = TDSCatalog('https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_onedeg/catalog.xml')
latest_coll = new_gfs.datasets[2]

In [3]:
ds = xr.open_dataset(latest_coll.access_urls['OPENDAP'])
ds = ds.metpy.parse_cf(ds)

# Fosberg Wildfire Index

<p>Represents the potential for wildfires based on a combination of relative humidity, wind, and temperature.
    It does not take into consideration non-meteorological factors such as fuels. The literature suggests values greater than 40 or 50,
    is when fire weather concerns exist. A value of 100 is achieved when the wind is equal to or greater than 30 mph and the
    relative humidity is closer to 0.
</p>

- [Calculation from the University of Washington](https://a.atmos.washington.edu/wrfrt/descript/definitions/fosbergindex.html)
- [Additional Explanation from NOAA's Storm Prediction Center](https://www.spc.noaa.gov/exper/firecomp/INFO/fosbinfo.html)
- [Operational Model Performance Research](https://www.spc.noaa.gov/publications/jirak/modelfwf.pdf)

In [4]:
def calculate_ffwi(temperature, wind_speed, relative_humidity_percent):   
    # Calculate moisture content (m) based on relative humidity (h) in percent
    m = np.where(relative_humidity_percent < 10,
                 0.03229 + 0.281073 * relative_humidity_percent - 0.000578 * relative_humidity_percent * temperature,
                 np.where((relative_humidity_percent >= 10) & (relative_humidity_percent <= 50),
                          2.22749 + 0.160107 * relative_humidity_percent - 0.01478 * temperature,
                          21.0606 + 0.005565 * relative_humidity_percent ** 2 - 0.00035 * relative_humidity_percent * temperature - 0.483199 * relative_humidity_percent))
    
    # Calculate moisture damping coefficient (n)
    n = 1 - 2 * (m / 30) + 1.5 * (m / 30) ** 2 - 0.5 * (m / 30) ** 3
    
    # Calculate FFWI
    fwi = n * np.sqrt(1 + wind_speed ** 2) / 0.3002
    
    # Ensure the FFWI is within the valid range [0, 100]
    fwi = np.maximum(0, np.minimum(100, fwi))
    
    return fwi

In [75]:
def update_fosberg(domain, time_index):
    t2m = ds.Temperature_height_above_ground.isel(time1=time_index).sel(height_above_ground3=2.0)
    t2m = t2m * units('K')
    t2m = t2m.metpy.convert_units('degF')

    rh2m = ds.Relative_humidity_height_above_ground.isel(time1=time_index).sel(height_above_ground4=2.0)

    u = ds['u-component_of_wind_height_above_ground'].isel(time1=time_index).sel(height_above_ground2=10.0)
    v = ds['u-component_of_wind_height_above_ground'].isel(time1=time_index).sel(height_above_ground2=10.0)
    s10m = mpcalc.wind_speed(u, v)
    s10m = s10m.metpy.convert_units('mph')

    ffwi = calculate_ffwi(t2m.values, s10m.values, rh2m.values)

    valid_time = (ds.time1.isel(time1=time_index))
    converted_time = np.datetime_as_string(valid_time, unit='m')
    
    cyclic_data, cyclic_lons = add_cyclic_point(ffwi, coord=ds.lon)
    
    fig = plt.figure(figsize=(10, 8))
    fig.canvas.toolbar_visible = False
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES.with_scale('110m'), linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE.with_scale('110m'))
    ax.add_feature(cfeature.OCEAN.with_scale('110m'), facecolor='lightblue', zorder=50, edgecolor='k')
    ax.add_feature(cfeature.BORDERS.with_scale('110m'), linewidth=0.5)
    ax.set_global() if not domain else ax.set_extent(domain)

    data = ax.contourf(cyclic_lons, ds.lat, cyclic_data, levels=[i for i in range(40, 101, 5)],
                       transform=ccrs.PlateCarree(), zorder=0, cmap='Oranges')
    
    plt.colorbar(data, fraction=0.02, pad=0.04)
    ax.set_title(f"Wildfire Favorability", loc='left', fontsize=14, fontweight='bold')
    ax.text(1, 1.01, f'Valid Time: {converted_time}', transform=ax.transAxes, fontsize=12, 
            verticalalignment='bottom', horizontalalignment='right')
    ax.text(0.99, 0.02, f"GFS Cycle: {np.datetime_as_string(ds.reftime.values, unit='m')}", fontsize=8, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9),
            ha='right', va='bottom', transform=ax.transAxes, zorder=60)
    fig.tight_layout()
    plt.show()

In [76]:
# Build the domain dropdown
domain = widgets.Dropdown(
    options=[('Global', []), ('North America', [-130, -65, 22, 60]),
             ('Europe', [-30, 60, 30, 70]), ('Australia', [110, 180, -45, -5]),
             ('South America', [-110, -25, -40, 10])],
    value=[],
    description='Domain:',
)

# Define the time index slider
time_slider = widgets.IntSlider(
    min=0,
    max=len(ds['time1'].values),
    step=1,
    description='Frame:',
    continuous_update=False,  # Update only when slider is released
    layout=widgets.Layout(width='50%')
)

# Create an interactive widget
interactive_plot = widgets.interactive(update_fosberg, domain=domain, time_index=time_slider)

# Display the interactive widget
display(interactive_plot)

interactive(children=(Dropdown(description='Domain:', options=(('Global', []), ('North America', [-130, -65, 2…